<a href="https://colab.research.google.com/github/anahimaciel/Networks-in-cities/blob/main/RadiusCalculator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd

from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame
import geopy.distance

In [ ]:
#read df
df = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/Caucaia/filtered_cdr.txt",usecols=[1,2,3,4,5])
df

,date,time,user_from,user_to,antenna
0,2013-04-16,11:20:40,CD933F79E567B215ACE12699BAFE7378,DA64FE0973FB116B6D18016DE360889D,3262
1,2013-04-16,15:18:55,AB60C0781BEC5EB9BEAC0E492A1D7FC3,B1D6C71B5201DC77ABB36855B7509722,3853
2,2013-04-16,15:28:53,AB60C0781BEC5EB9BEAC0E492A1D7FC3,0BC3EAE9D47DE038442A082365F6FD5F,3853
3,2013-04-16,18:30:06,9D77EA5E65FB391CCB21F3BA6E51A313,64A1EA8369CB1B561635CFE799D0DF77,3853
4,2013-04-16,12:21:04,9D77EA5E65FB391CCB21F3BA6E51A313,358EC95E9F736FBA1D01593A05049BF6,3853
...,...,...,...,...,...
12918825,2013-04-03,15:51:43,E0A2162D07B0FE9210A464D514218145,7451F737989B09DD2C00A10D17E45896,3191
12918826,2013-04-03,19:04:34,214F5460EA51408F82C2E77A3FB1D607,A49BA005920B8BA24BEE40F455B69B9B,3852
12918827,2013-04-03,17:21:58,64DD6D0023CCD79CF782B89B841C8834,DA509B7F4B9B53EF84B3EC98ED2CA902,3222
12918828,2013-04-03,11:15:00,8A0BC9F7915098CFEADDE8241D93848A,1C7843341899AA596376AAA5CE942FBD,3823


In [ ]:
#read antennas files
map = pd.read_csv("/content/drive/MyDrive/Networks in cities/Dados/Caucaia/antennas_Caucaia.txt", sep=';')
map

,CELLID,UF,DDD,LAT,LONG,CITY,MUNICIPALITY
0,7537,CE,85,-3.73667,-38.74306,Caucaia,Microrregião de Fortaleza
1,7498,CE,85,-3.73389,-38.65722,Caucaia,Microrregião de Fortaleza
2,7478,CE,85,-3.76639,-38.63389,Caucaia,Microrregião de Fortaleza
3,7529,CE,85,-3.68083,-38.66083,Caucaia,Microrregião de Fortaleza
4,7517,CE,85,-3.69444,-38.62250,Caucaia,Microrregião de Fortaleza
...,...,...,...,...,...,...,...
111,3191,CE,85,-3.73389,-38.65722,Caucaia,Microrregião de Fortaleza
112,1603,CE,85,-3.74214,-38.61220,Caucaia,Microrregião de Fortaleza
113,3173,CE,85,-3.76639,-38.63389,Caucaia,Microrregião de Fortaleza
114,3831,CE,85,-3.62861,-38.72778,Caucaia,Microrregião de Fortaleza


# Compute home locations

In [ ]:
#convert to datetime object
#df['date']=pd.to_datetime(df['date'])
#create column with days of the week
df['day_week']=df['date'].dt.day_name()

#create dataframe with calls in weekends and nights
night_df=df[(df['day_week']=='Saturday') | (df['day_week']=='Sunday') | (df['time']> '19:00:00') | (df['time']< '06:00:00')]
night_df

,date,time,user_from,user_to,antenna,day_week
5,2013-04-16,19:07:11,9D77EA5E65FB391CCB21F3BA6E51A313,FF099411110E8B3DE067D51EEA021C14,3853,Tuesday
8,2013-04-16,19:44:02,62905216A1E565F3FEBE078BB9536CED,9B0AF7C3AE44AE048DECC9F48ED544ED,3171,Tuesday
9,2013-04-16,22:39:34,62905216A1E565F3FEBE078BB9536CED,9B0AF7C3AE44AE048DECC9F48ED544ED,3171,Tuesday
21,2013-04-16,20:20:10,BD2337B9B93D7808EA4307977FF898DD,E47FC4C1F09A260E3CD8DC8A5CE77379,3853,Tuesday
23,2013-04-16,20:09:52,0E73AA3D46812B10DDD50C55E3AA755D,695C84609CB5379D59DB01881A20133C,3832,Tuesday
...,...,...,...,...,...,...
12918792,2013-04-03,20:37:42,C5DDA8EF72480F4DF02BBC893BF68353,C43925191E52644B02983928A1BC24B6,3192,Wednesday
12918799,2013-04-03,21:19:42,7E2FF3F4BAB4FED6C8728A7308A5D2F8,EDE359F5A0A10B02C28A6D6B03960F3D,3223,Wednesday
12918803,2013-04-03,20:14:20,E523F297065CC29660A8A91B3D4717BD,B3FD4FA893D66C8B9E0DC15DA9109314,3183,Wednesday
12918815,2013-04-03,05:11:22,EE64D291BAC6C680C1F84235EB493F56,153C93B31F8538DDFF626109318A20FB,3232,Wednesday


In [ ]:
#create dataframe with data necessary to compute places of residence
residence_df=night_df.groupby(['user_from','antenna']).size().reset_index(name='time')
residence_df

,user_from,antenna,time
0,0000489E396F11608F0DF012E4FAB9B8,1103,2
1,00008112363605FFC0A6971A16D98128,3823,1
2,0000851C574768CA2BB1156851A3171C,3872,1
3,000091351228C640CD3896DF721F3FAB,3273,1
4,0000C3C0AA55DAE28C4C73978E57E953,1103,1
...,...,...,...
1058950,FFFFC496AB4D2D6CF1ABAE1C4DF6DFE3,1772,1
1058951,FFFFC5D625F2527647BD4D859AB95B37,3191,4
1058952,FFFFC5D625F2527647BD4D859AB95B37,3264,2
1058953,FFFFC5D625F2527647BD4D859AB95B37,3873,1


In [ ]:
#compute places of residence
#drop locations other than the first
residence_df=residence_df.sort_values(['user_from','time'],ascending=[True,False]).drop_duplicates('user_from')
#only keep users whose first location complies with certain parameters (more than half of the calls of the user)
number_calls_map=residence_df['user_from'].map(df.groupby('user_from').size())
filter=(residence_df['time']>number_calls_map/2)
residence_df=residence_df[filter]

#include geographical location of antennas in residence dataframe
map=map.rename(columns={"CELLID": "antenna"})
residence_df=pd.merge(residence_df,map,on="antenna",how="left")

#delete unnecessary columns
residence_df=residence_df.drop(['time','UF','DDD','CITY','MUNICIPALITY'],axis=1)

residence_df.sort_values('antenna')

,user_from,antenna,time
1,00008112363605FFC0A6971A16D98128,3823,1
2,0000851C574768CA2BB1156851A3171C,3872,1
9,0000F8778334D32580C06AF4C176E162,3191,1
12,00018C8646F523A687A8387E76A47295,1601,1
13,00018D858C1D7803B20F7B0DF28E9211,1102,1
...,...,...,...
1058931,FFFF6761884073022FB5DFE1B6F54DE3,892,1
1058932,FFFF73B828960B2C39C560BBA1B93B4C,2422,3
1058948,FFFFBBF11C6DCD793A020564A9CF6FFE,893,1
1058951,FFFFC5D625F2527647BD4D859AB95B37,3191,4


In [ ]:
#extract from df users whose place of residence were not found (useful for next step)
filter=df['user_from'].isin(residence_df['user_from']) & df['user_to'].isin(residence_df['user_from'])
filtered_df=df[filter]
filtered_df

,date,time,user_from,user_to,antenna,day_week
164,2013-04-16,19:11:49,DE4404DC86F849004FE3A1BC1ADAA5E2,428C2F561814A45AA4A605486726DB34,3851,Tuesday
247,2013-04-16,11:15:35,069706C99389AB545445FEED90EEE489,82DEDBFC305A30F11E790E9AAEDE0979,3262,Tuesday
716,2013-04-16,20:54:32,F1000185B6F729C0E5E5974883119B66,623A6EADAD853ED1B0FEE4A717CAA647,3213,Tuesday
877,2013-04-16,22:46:22,D766D047E2E1B8AEE9B86FBE7CD2477F,6FC0AA1BB9433E0AA5A2C8D2673A3278,1772,Tuesday
908,2013-04-16,19:29:24,629D1378C07623B991970AA204F76657,F0E58EF4EA5E924C6A182DE6D62D1FF9,3822,Tuesday
...,...,...,...,...,...,...
12918270,2013-04-03,23:48:13,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday
12918301,2013-04-03,10:47:32,9A948ECDD89B8E74DD34631BEDA88D75,0D473CFA2F240691C1BB872E90B7DF35,3183,Wednesday
12918414,2013-04-03,23:50:32,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday
12918616,2013-04-03,08:49:19,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday


# Calculate average radius

In [ ]:
#dicts with coordinates
dict_lat = residence_df.set_index('user_from')['LAT'].to_dict()
dict_long = residence_df.set_index('user_from')['LONG'].to_dict()

#function to calculate distance between residences of caller and receiver for each call
def get_distance(row):
    coords_caller=(dict_lat[row.user_from],dict_long[row.user_from])                #deixar mais rapido: agrupar por pessoa q ta ligando (n sei se ajudaria mt)
    coords_receiver=(dict_lat[row.user_to],dict_long[row.user_to])
    return geopy.distance.distance(coords_caller,coords_receiver).km

#apply function to filtered_df
filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)
filtered_df

<ipython-input-19-bd0efc1160f5>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['distance'] = filtered_df.apply(get_distance, axis=1)


,date,time,user_from,user_to,antenna,day_week,distance
164,2013-04-16,19:11:49,DE4404DC86F849004FE3A1BC1ADAA5E2,428C2F561814A45AA4A605486726DB34,3851,Tuesday,0.000000
247,2013-04-16,11:15:35,069706C99389AB545445FEED90EEE489,82DEDBFC305A30F11E790E9AAEDE0979,3262,Tuesday,0.000000
716,2013-04-16,20:54:32,F1000185B6F729C0E5E5974883119B66,623A6EADAD853ED1B0FEE4A717CAA647,3213,Tuesday,8.448165
877,2013-04-16,22:46:22,D766D047E2E1B8AEE9B86FBE7CD2477F,6FC0AA1BB9433E0AA5A2C8D2673A3278,1772,Tuesday,0.000000
908,2013-04-16,19:29:24,629D1378C07623B991970AA204F76657,F0E58EF4EA5E924C6A182DE6D62D1FF9,3822,Tuesday,1.358800
...,...,...,...,...,...,...,...
12918270,2013-04-03,23:48:13,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday,12.338536
12918301,2013-04-03,10:47:32,9A948ECDD89B8E74DD34631BEDA88D75,0D473CFA2F240691C1BB872E90B7DF35,3183,Wednesday,24.055581
12918414,2013-04-03,23:50:32,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday,12.338536
12918616,2013-04-03,08:49:19,9A948ECDD89B8E74DD34631BEDA88D75,B6B8EBA0C0402F5E019BC41197D86B49,3183,Wednesday,12.338536


In [ ]:
#drop calls to the same receiver (nao sei pq)
groups=filtered_df.groupby(['user_from','user_to'])
aux=groups.first()
aux=aux.groupby('user_from')['distance'].sum().to_frame(name='total_distance_without_alpha')

#calculate average radius for each user
radii=filtered_df.groupby("user_from").agg({'distance':['sum','size'],'user_to':'nunique'})
radii.columns=['total_distance','n_calls','contacts']
radii['avg_dist_by_calls']=radii['total_distance']/radii['n_calls']
radii['avg_radius']=radii['total_distance']/radii['contacts']
radii['total_distance_without_alpha']=aux['total_distance_without_alpha']
radii['avg_radius_without_alpha']=aux['total_distance_without_alpha']/radii['contacts']
radii

,total_distance,n_calls,contacts,avg_dist_by_calls,avg_radius,total_distance_without_alpha,avg_radius_without_alpha
user_from,,,,,,,
00008112363605FFC0A6971A16D98128,5.459592,1,1,5.459592,5.459592,5.459592,5.459592
0000851C574768CA2BB1156851A3171C,1.427975,1,1,1.427975,1.427975,1.427975,1.427975
0001C104EA75AFE78EDF1392EBBEDDAC,17.742308,5,1,3.548462,17.742308,3.548462,3.548462
00037AAF3500E0CAC6F58A0F66B5D973,7.446944,1,1,7.446944,7.446944,7.446944,7.446944
0003DA7448B9BA48C5DF506C14E3E078,0.000000,6,2,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
FFFB450B3EF81CF9FEA201402530CDD6,20.131646,2,1,10.065823,20.131646,10.065823,10.065823
FFFBB51DAA4539DB9A84C7981C6FD248,0.000000,1,1,0.000000,0.000000,0.000000,0.000000
FFFF0D4939E61BDC960B3E4BED330B5D,3.548462,1,1,3.548462,3.548462,3.548462,3.548462


In [ ]:
for i in radii.columns:
  print(radii.agg({i:['max','min','mean']}))

      total_distance
max      3970.919519
min         0.000000
mean        9.476664
         n_calls
max   732.000000
min     1.000000
mean    2.922581
       contacts
max   25.000000
min    1.000000
mean   1.248142
      avg_dist_by_calls
max           30.300738
min            0.000000
mean           4.298941
       avg_radius
max   2382.643188
min      0.000000
mean     6.755228
      total_distance_without_alpha
max                     345.894808
min                       0.000000
mean                      5.214380
      avg_radius_without_alpha
max                  30.300738
min                   0.000000
mean                  4.321585


In [ ]:
#saving results
residence_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/Caucaia/home_locations.txt")
filtered_df.to_csv("/content/drive/MyDrive/Networks in cities/Dados/Caucaia/filtered_cdr_2.txt")

#for the networks
edges=groups.size().reset_index(name='weight')
edges.to_csv("/content/drive/MyDrive/Networks in cities/Dados/Caucaia/edges.txt")